# VGG16 for Age Estimation

## Import Data

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#import images from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import glob
bucket0 = glob.glob('/content/drive/MyDrive/Colab Notebooks/StocktonImagesBucketed/0/*.*')
bucket1 = glob.glob('/content/drive/MyDrive/Colab Notebooks/StocktonImagesBucketed/1/*.*')
bucket2 = glob.glob('/content/drive/MyDrive/Colab Notebooks/StocktonImagesBucketed/2/*.*')
bucket3 = glob.glob('/content/drive/MyDrive/Colab Notebooks/StocktonImagesBucketed/3/*.*')
bucket4 = glob.glob('/content/drive/MyDrive/Colab Notebooks/StocktonImagesBucketed/4/*.*')
bucket5 = glob.glob('/content/drive/MyDrive/Colab Notebooks/StocktonImagesBucketed/5/*.*')
bucket6 = glob.glob('/content/drive/MyDrive/Colab Notebooks/StocktonImagesBucketed/6/*.*')
bucket7 = glob.glob('/content/drive/MyDrive/Colab Notebooks/StocktonImagesBucketed/7/*.*')
bucket8 = glob.glob('/content/drive/MyDrive/Colab Notebooks/StocktonImagesBucketed/8/*.*')

data = []
labels = []
for i in bucket0:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('0')

for i in bucket1:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('0')

for i in bucket2:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('0')      

for i in bucket3:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('0')

for i in bucket4:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('0')

for i in bucket5:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('1')   

for i in bucket6:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('1')

for i in bucket7:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('1')

for i in bucket8:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('1')        

data = np.array(data)
labels = np.array(labels)

In [40]:
#Split data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=4)

In [41]:
print(X_train.shape)
print(y_train.shape)

(2088, 224, 224, 3)
(2088,)


In [42]:
print(X_test.shape)
print(y_test.shape)

(523, 224, 224, 3)
(523,)


In [43]:
#Normalize data
#X_train = X_train.astype('float32') / 255
#X_test = X_test.astype('float32') / 255

#Convert labels to categorical
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes = 2)
y_test = to_categorical(y_test, num_classes = 2)

In [44]:
#augment data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=40, width_shift_range=0.3, height_shift_range=0.3, shear_range=0.3, horizontal_flip=True, fill_mode="nearest")
train_generator = train_datagen.flow(X_train, y_train, batch_size=1)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(X_test, y_test, batch_size=1)

## Model

In [45]:
from keras.applications import VGG16
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [46]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [47]:
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model

x = vgg_model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x) # Binary classification
transfer_model = Model(inputs=vgg_model.input, outputs=x)

In [48]:
# Freeze four convolution blocks
for layer in transfer_model.layers[:15]:
    layer.trainable = False

# Make sure you have frozen the correct layers
for i, layer in enumerate(transfer_model.layers):
    print(i, layer.name, layer.trainable)

0 input_3 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 flatten_7 True
20 dense_21 True
21 dropout_7 True
22 dense_22 True
23 dense_23 True


In [ ]:
#compiling and fitting model WITHOUT augmented data
from keras import optimizers

learning_rate= 0.001
transfer_model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=learning_rate), metrics=["accuracy"])
history = transfer_model.fit(X_train, y_train, batch_size = 16, epochs=50, validation_data=(X_test,y_test))

In [49]:
#compiling and fitting model WITH augmented data
from keras import optimizers

learning_rate= 0.0001
transfer_model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=learning_rate), metrics=["accuracy"])
history = transfer_model.fit_generator(train_generator, validation_data = test_generator, epochs=10)

Epoch 1/10


<ipython-input-49-2c1e5f607210>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = transfer_model.fit_generator(train_generator, validation_data = test_generator, epochs=10)


InvalidArgumentError: ignored

In [ ]:
#Plot accuracy and loss
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Transfer Learning Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()